# Compute FLOPS

In [25]:
import torch
import numpy as np
import sys
sys.path.append('../')

from model_utils import load_model

# MODEL_NAME = 'vgg16_imagenet'
# MODEL_NAME = 'resnet50_imagenet'
# MODEL_NAME = 'faster_rcnn_vgg16'

sys.path.append('/workspace/home/jgusak/maxvol_objects/facebook_frcnn/')
import maskrcnn_benchmark
MODEL_NAME = 'faster_rcnn_resnet50'
# MODEL_NAME = 'faster_rcnn_resnet50_fpn'

model = load_model(MODEL_NAME)
# model = torch.load('/workspace/raid/data/eponomarev/results/musco_voc/vgg16_fasterrcnn/tucker2/rank_selection:nx/ranks:/1.77x_layer_groups:1/vgg-16.bb')

# compressed_model =  torch.load('/workspace/raid/data/jgusak/backbone_iter_0-0.pth')
# compressed_model = torch.load('/workspace/raid/data/eponomarev/results/musco_voc/vgg16_fasterrcnn/tucker2/rank_selection:nx/ranks:/1.77x_layer_groups:1/backbone_iter:0-1.pth')
compressed_model = torch.load("/workspace/raid/data/eponomarev/frcnn_models_hachiko2/resnet50_fasterrcnn/cp3/STEP_INV_LBL3_cp3_5/vbmf_0.9/0/2/compressed_frcnn_model.backbone")
# compressed_model = torch.load("/workspace/raid/data/eponomarev/frcnn_models_hachiko2/resnet50_fasterrcnn/cp3/STEP_INV_LBL3_cp3_10/vbmf_0.9/0/2/compressed_frcnn_model_backbone.pth")

model

Sequential(
  (body): ResNet(
    (stem): StemWithFixedBatchNorm(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
    )
    (layer1): Sequential(
      (0): BottleneckWithFixedBatchNorm(
        (downsample): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): FrozenBatchNorm2d()
        )
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): FrozenBatchNorm2d()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): FrozenBatchNorm2d()
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): FrozenBatchNorm2d()
      )
      (1): BottleneckWithFixedBatchNorm(
        (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): FrozenBatchNorm2d()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stri

In [26]:
compressed_model

Sequential(
  (body): ResNet(
    (stem): StemWithFixedBatchNorm(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
    )
    (layer1): Sequential(
      (0): BottleneckWithFixedBatchNorm(
        (downsample): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): FrozenBatchNorm2d()
        )
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): FrozenBatchNorm2d()
        (conv2): Sequential(
          (conv2-0): Conv2d(64, 53, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (conv2-1): Conv2d(53, 53, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=53, bias=False)
          (conv2-2): Conv2d(53, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (bn2): FrozenBatchNorm2d()
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): FrozenBatchNorm2d()


In [27]:
import torch
from model_utils import  compute_model_flops, \
get_input_shapes_vgg, get_input_shapes_resnet, \
get_input_shapes_faster_rcnn_vgg, \
get_input_shapes_faster_rcnn_resnet50, get_input_shapes_faster_rcnn_resnet50_fpn

device = 'cuda'
x = torch.randn(1, 3, 224, 224).to(device)

if MODEL_NAME == 'vgg16_imagenet':
    input_shapes_dict = get_input_shapes_vgg(model, x)
    input_shapes_dict_cm = get_input_shapes_vgg(compressed_model, x)

elif MODEL_NAME == 'resnet50_imagenet':
    input_shapes_dict = get_input_shapes_resnet50(model, x)
    input_shapes_dict_cm = get_input_shapes_resnet50(compressed_model, x)
    
elif MODEL_NAME == 'faster_rcnn_vgg16':
    input_shapes_dict = get_input_shapes_faster_rcnn_vgg(model, x)
    input_shapes_dict_cm = get_input_shapes_faster_rcnn_vgg(compressed_model, x)
    
elif MODEL_NAME == 'faster_rcnn_resnet50':
    input_shapes_dict = get_input_shapes_faster_rcnn_resnet50(model, x)
    input_shapes_dict_cm = get_input_shapes_faster_rcnn_resnet50(compressed_model, x)
    
elif MODEL_NAME == 'faster_rcnn_resnet50_fpn':
    input_shapes_dict = get_input_shapes_faster_rcnn_resnet50_fpn(model, x)
    input_shapes_dict_cm = get_input_shapes_faster_rcnn_resnet50_fpn(compressed_model, x)
    


In [28]:
flops_m, conv_flops_m, _ = compute_model_flops(model, input_shapes_dict)
flops_cm, conv_flops_cm, _ = compute_model_flops(compressed_model, input_shapes_dict_cm)

for flops, model_title in zip([flops_m, flops_cm],
                              ['Initial model', 'Compressed model']):
    print('{}, conv: {}({}), fc: {}({})'.format(model_title,
                                                flops['conv'],
                                                np.round(flops['conv']/sum(flops.values()), decimals = 2),
                                                flops['fc'],
                                                np.round(flops['fc']/sum(flops.values()), decimals = 2)))

print('\n FLOPS speed-up, conv: {}, total: {}'.format(flops_m['conv']/flops_cm['conv'],
                                                   sum(flops_m.values())/sum(flops_cm.values())
                                                  ))
if flops_cm['fc'] > 0:
    print(' FLOPS speed-up, fc: {}, total: {}'.format(flops_m['fc']/flops_cm['fc'],
                                                      sum(flops_m.values())/sum(flops_cm.values())
                                                     ))

Initial model, conv: 3123757056(1.0), fc: 0(0.0)
Compressed model, conv: 1919911728(1.0), fc: 0(0.0)

 FLOPS speed-up, conv: 1.6270316027779377, total: 1.6270316027779377


In [20]:
sum([kv[1] for kv in conv_flops_cm.items() if kv[0].startswith('2.')])

0

In [27]:
for k,v in conv_flops_m.items():
    if k != '0':
        rate = conv_flops_m['{}'.format(k)]/sum([kv[1] for kv in conv_flops_cm.items()
                                                 if kv[0].startswith('{}.'.format(k))])
        print(k, rate)

2 1.809098493399421
5 1.787085514834206
7 1.7712645196939303
10 1.787085514834206
12 1.7712645196939303
14 1.7712645196939303
17 1.7736800559327004
19 1.7712645196939303
21 1.7712645196939303
24 1.7712645196939303
26 1.7712645196939303
28 1.7712645196939303


In [35]:
conv_flops_cm

{'body.stem.conv1': 118013952,
 'body.layer1.0.downsample.0': 51380224,
 'body.layer1.0.conv1': 12845056,
 'body.layer1.0.conv2.conv2-0': 26894336,
 'body.layer1.0.conv2.conv2-1': 3782016,
 'body.layer1.0.conv2.conv2-2': 26894336,
 'body.layer1.0.conv3': 51380224,
 'body.layer1.1.conv1': 51380224,
 'body.layer1.1.conv2.conv2-0': 26894336,
 'body.layer1.1.conv2.conv2-1': 3782016,
 'body.layer1.1.conv2.conv2-2': 26894336,
 'body.layer1.1.conv3': 51380224,
 'body.layer1.2.conv1': 51380224,
 'body.layer1.2.conv2.conv2-0': 26894336,
 'body.layer1.2.conv2.conv2-1': 3782016,
 'body.layer1.2.conv2.conv2-2': 26894336,
 'body.layer1.2.conv3': 51380224,
 'body.layer2.0.downsample.0': 102760448,
 'body.layer2.0.conv1': 25690112,
 'body.layer2.0.conv2.conv2-0': 27897856,
 'body.layer2.0.conv2.conv2-1': 1961568,
 'body.layer2.0.conv2.conv2-2': 27897856,
 'body.layer2.0.conv3': 51380224,
 'body.layer2.1.conv1': 51380224,
 'body.layer2.1.conv2.conv2-0': 27897856,
 'body.layer2.1.conv2.conv2-1': 196156

In [7]:
compute_model_flops(compressed_model, input_shapes_dict_cm, verbose = True)


0 0.01
2.2-0 0.02
2.2-1 0.09
2.2-2 0.02
5.5-0 0.0
5.5-1 0.04
5.5-2 0.01
7.7-0 0.02
7.7-1 0.09
7.7-2 0.02
10.10-0 0.0
10.10-1 0.04
10.10-2 0.01
12.12-0 0.02
12.12-1 0.09
12.12-2 0.02
14.14-0 0.02
14.14-1 0.09
14.14-2 0.02
17.17-0 0.0
17.17-1 0.04
17.17-2 0.01
19.19-0 0.02
19.19-1 0.09
19.19-2 0.02
21.21-0 0.02
21.21-1 0.09
21.21-2 0.02
24.24-0 0.0
24.24-1 0.02
24.24-2 0.0
26.26-0 0.0
26.26-1 0.02
26.26-2 0.0
28.28-0 0.0
28.28-1 0.02
28.28-2 0.0



{'conv': 8670181520, 'fc': 0}

In [36]:
conv_flops_m

{'body.stem.conv1': 118013952,
 'body.layer1.0.downsample.0': 51380224,
 'body.layer1.0.conv1': 12845056,
 'body.layer1.0.conv2.conv2-0': 2609152,
 'body.layer1.0.conv2.conv2-1': 4769856,
 'body.layer1.0.conv2.conv2-2': 2609152,
 'body.layer1.0.conv3': 51380224,
 'body.layer1.1.conv1': 51380224,
 'body.layer1.1.conv2.conv2-0': 2609152,
 'body.layer1.1.conv2.conv2-1': 4769856,
 'body.layer1.1.conv2.conv2-2': 2609152,
 'body.layer1.1.conv3': 51380224,
 'body.layer1.2.conv1': 51380224,
 'body.layer1.2.conv2.conv2-0': 2609152,
 'body.layer1.2.conv2.conv2-1': 4769856,
 'body.layer1.2.conv2.conv2-2': 2609152,
 'body.layer1.2.conv3': 51380224,
 'body.layer2.0.downsample.0': 102760448,
 'body.layer2.0.conv1': 25690112,
 'body.layer2.0.conv2.conv2-0': 2709504,
 'body.layer2.0.conv2.conv2-1': 5143824,
 'body.layer2.0.conv2.conv2-2': 2709504,
 'body.layer2.0.conv3': 51380224,
 'body.layer2.1.conv1': 51380224,
 'body.layer2.1.conv2.conv2-0': 2709504,
 'body.layer2.1.conv2.conv2-1': 5143824,
 'body